In [9]:
from random import random
from random import seed
from math import exp

In [10]:
def initialized(n_inputs,n_hidden,n_outputs):
    network=list()
    hidden_layer=[{'weights':[random()for i in range(n_inputs+1)]}for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer=[{'weights':[random()for i in range(n_hidden+1)]}for i in range(n_outputs)]
    network.append(output_layer)
    return network

seed(1)
network=initialized(2,1,2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [11]:
def activate(weights,inputs):
    activation=weights[-1]
    for i in range(len(weights)-1):
        activation +=weights[i]*inputs[i]
    return activation

def transfer(activation):
    return 1/(1+exp(-activation))


def forward(network,row):
    inputs=row
    for layer in network:
        new_input=[]
        for neuron in layer:
            activation=activate(neuron['weights'],inputs)
            neuron['outputs']=transfer(activation)
            new_input.append(neuron['outputs'])
        inputs=new_input
    return inputs

network=[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row=[1,0,None]
outputs=forward(network,row)
print(outputs)

[0.6629970129852887, 0.7253160725279748]


In [12]:
def tranfer_deri(outputs):
    return outputs*(1-outputs)

def backward(network,expected):
    for i in reversed(range(len(network))):
        layer=network[i]
        errors=list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error=0
                for neuron in network[i+1]:
                    error +=(neuron['weights'][j]*neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron=layer[j]
                errors.append(neuron['outputs']-expected[j])
        for j in range(len(layer)):
            neuron=layer[j]
            neuron['delta']=errors[j]*tranfer_deri(neuron['outputs'])

network=[[{'outputs':0.71,'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
[{'outputs':0.65,'weights': [0.2550690257394217, 0.49543508709194095]}, {'outputs':0.62,'weights': [0.4494910647887381, 0.651592972722763]}]]
expected=[0,1]
backward(network,expected)
for layer in network:
    print(layer)

[{'outputs': 0.71, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.0005196306262542487}]
[{'outputs': 0.65, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': 0.14787499999999998}, {'outputs': 0.62, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': -0.089528}]


In [21]:
def update(network,row,lr):
    for i in range(len(network)):
        inputs=row[:-1]
        if i != 0:
            inputs=[neuron['outputs']for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] -=lr*neuron['delta']*inputs[j]
            neuron['weights'][-1] -=lr*neuron['delta']

def train_network(network,train,lr,ep,n_outputs):
    for epoch in range(ep):
        sum_error=0
        for row in train:
            outputs=forward(network,row)
            expected=[0 for i in range(n_outputs)]
            expected[row[-1]]=1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward(network,expected)
            update(network,row,lr)
        print(f"epoch {epoch} lr {lr} Mse :{sum_error}")


# Test training backprop algorithm
seed(1)
dataset = [[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialized(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 3, n_outputs) # 3 ki jagha 300000 daalna
output_string = ""
for layer in network:
  for neuron in layer:
      output_string += str(neuron) + "\n"
print(output_string)


epoch 0 lr 0.5 Mse :2.521079382881022
epoch 1 lr 0.5 Mse :2.4093799056984917
epoch 2 lr 0.5 Mse :2.32145670739353
{'weights': [0.1153246121879374, 0.8354470284381753, 0.7391084055828353], 'outputs': 0.842993624934674, 'delta': -0.006138040417544435}
{'weights': [0.21815242923559697, 0.45934085194038776, 0.3716239403048077], 'outputs': 0.7439943314137906, 'delta': 0.011811071404330655}
{'weights': [0.46898606192670256, 0.6264198253517977, -0.1428812253801669], 'outputs': 0.652320642100105, 'delta': -0.07885312973026624}
{'weights': [-0.14357367575969715, 0.6824066438871634, 0.20487654877540173], 'outputs': 0.6811424167715947, 'delta': 0.1479355674521029}



In [22]:
def predict(network,row):
    outputs=forward(network,row)
    return outputs.index(max(outputs))

for row in dataset:
    prediction=predict(network,row)
    print(f"Actual {row[-1]} Predicted {prediction}")

Actual 0 Predicted 0
Actual 1 Predicted 0
Actual 1 Predicted 0
Actual 0 Predicted 0
